In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from global_import import *
from fm import fama_macbeth_reg_panel_nw

In [2]:
gkx = pd.read_csv(f'{DATA_FOLDER}/gkx_mlfactors.csv')

In [3]:
gkx['date']=pd.to_datetime(gkx['DATE'], format='%Y%m%d')+MonthEnd(0)

In [10]:
ret_decomp_dcf5 = pd.read_csv(f'{DATA_FOLDER}/decomp_dcf5y_PLS_1980_2019.csv', index_col=0)
ret_decomp_dcf5['jdate'] = pd.to_datetime(ret_decomp_dcf5.jdate)
ret_decomp_dcf5 = ret_decomp_dcf5.replace([np.inf, -np.inf], np.nan)

In [11]:
ret_decomp_dcf5['error_60'] = ret_decomp_dcf5['r_t+60'] - ret_decomp_dcf5['r*_t+60'] - ret_decomp_dcf5['r_chg+60']  - ret_decomp_dcf5['r_unchg+60'] 
ret_decomp_dcf5[['r_t+60', 'r*_t+60', 'r_chg+60', 'r_unchg+60', 'error_60']].describe()

,r_t+60,r*_t+60,r_chg+60,r_unchg+60,error_60
count,1.251811e+06,632273.000000,632106.000000,1.251539e+06,632104.000000
mean,-1.874065e-02,0.519025,-0.166237,3.499541e-02,-0.032174
std,1.111623e+00,1.368797,1.527590,5.136348e-02,0.536504
min,-1.874448e+01,-9.438013,-12.349693,-2.602439e-01,-16.716173
25%,-4.166773e-01,-0.197280,-1.002409,-1.083967e-03,-0.152535
50%,1.168368e-01,0.516434,-0.141979,2.877244e-02,0.035595
75%,6.282922e-01,1.232840,0.683163,6.710830e-02,0.187090
max,5.813533e+00,12.818337,12.004779,4.433868e-01,10.123112


In [12]:
mdf5 = gkx.merge(ret_decomp_dcf5, left_on=['date', 'permno'], 
                 right_on=['jdate', 'permno'], how='inner')

In [13]:
vals = [
 'mvel1',
 'beta',
 'idiovol',
 'indmom',
 'mom1m',
 'mom6m',
 'mom12m',
 'mom36m',
 'pricedelay',
 'age',
 'agr',
 'bm',
 'bm_ia',
 'cashpr',
 'cfp',
 'cfp_ia',
 'chatoia',
 'chcsho',
 'depr',
 'divi',
 'egr',
 'ep',
 'grcapx',
 'hire',
 'invest',
 'lev',
 'mve_ia',
 'orgcap',
 'pchcurrat',
 'pchdepr',
 'pctacc',
 'quick',
 'rd',
 'rd_mve',
 'roic',
 'secured',
 'sp',
 'tang',
 'tb',
 'cash',
 'chtx',
 'cinvest',
 'nincr',
 'roaq',
 'roavol',
 'roeq',
 'rsup',
 'ms',
 'ill',
 'std_turn',]

In [14]:
mdf5['ret'] = np.exp(mdf5['r_t+60'])-1
mdf5['ret_ct'] = np.exp(mdf5['r*_t+60']-1)
mdf5['ret_chg'] = np.exp(mdf5['r_chg+60'])

In [16]:
res = []
for var in vals:
    regdf = mdf5[['jdate', 'permno', 'ticker', 'ret', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'Ret ~ {var}, 5Yr-DCF')
    results = fama_macbeth_reg_panel_nw(regdf, xname=var, yname='ret')
#     print(results)
    res.append(results)
#     print('')

Ret ~ mvel1, 5Yr-DCF
Ret ~ beta, 5Yr-DCF
Ret ~ idiovol, 5Yr-DCF
Ret ~ indmom, 5Yr-DCF
Ret ~ mom1m, 5Yr-DCF
Ret ~ mom6m, 5Yr-DCF
Ret ~ mom12m, 5Yr-DCF
Ret ~ mom36m, 5Yr-DCF
Ret ~ pricedelay, 5Yr-DCF
Ret ~ age, 5Yr-DCF
Ret ~ agr, 5Yr-DCF
Ret ~ bm, 5Yr-DCF
Ret ~ bm_ia, 5Yr-DCF
Ret ~ cashpr, 5Yr-DCF
Ret ~ cfp, 5Yr-DCF
Ret ~ cfp_ia, 5Yr-DCF
Ret ~ chatoia, 5Yr-DCF
Ret ~ chcsho, 5Yr-DCF
Ret ~ depr, 5Yr-DCF
Ret ~ divi, 5Yr-DCF
Ret ~ egr, 5Yr-DCF
Ret ~ ep, 5Yr-DCF
Ret ~ grcapx, 5Yr-DCF
Ret ~ hire, 5Yr-DCF
Ret ~ invest, 5Yr-DCF
Ret ~ lev, 5Yr-DCF
Ret ~ mve_ia, 5Yr-DCF
Ret ~ orgcap, 5Yr-DCF
Ret ~ pchcurrat, 5Yr-DCF
Ret ~ pchdepr, 5Yr-DCF
Ret ~ pctacc, 5Yr-DCF
Ret ~ quick, 5Yr-DCF
Ret ~ rd, 5Yr-DCF
Ret ~ rd_mve, 5Yr-DCF
Ret ~ roic, 5Yr-DCF
Ret ~ secured, 5Yr-DCF
Ret ~ sp, 5Yr-DCF
Ret ~ tang, 5Yr-DCF
Ret ~ tb, 5Yr-DCF
Ret ~ cash, 5Yr-DCF
Ret ~ chtx, 5Yr-DCF
Ret ~ cinvest, 5Yr-DCF
Ret ~ nincr, 5Yr-DCF
Ret ~ roaq, 5Yr-DCF
Ret ~ roavol, 5Yr-DCF
Ret ~ roeq, 5Yr-DCF
Ret ~ rsup, 5Yr-DCF
Ret ~ ms, 5Yr-DCF

In [20]:
res_df  = pd.concat(res)
res_df = res_df[res_df.index != 'alpha_i_t']

In [21]:
res_df.head()

,count,mean,std_error,tstat,std_error_nw,tstat_nw
mvel1,444.0,7.930238e-09,4.882133e-09,1.624339,1.155778e-08,0.686139
beta,444.0,-1.512155e-01,1.552241e-02,-9.741752,3.623349e-02,-4.173363
idiovol,444.0,-4.312743e+00,2.204843e-01,-19.560314,5.104717e-01,-8.448545
indmom,444.0,1.433861e-01,5.430953e-02,2.640165,1.214449e-01,1.180668
mom1m,444.0,1.380587e-01,4.268855e-02,3.234092,6.725468e-02,2.052775


In [ ]:
res_ct = []
for var in vals:
    regdf = mdf5[['jdate', 'permno', 'ticker', 'ret_ct', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'{var}, retct')
    results = fama_macbeth_reg_panel_nw(regdf, xname=var, yname='ret_ct')
#     print(results)
    res_ct.append(results)
#     print('')

mvel1, retct
beta, retct
idiovol, retct
indmom, retct
mom1m, retct
mom6m, retct
mom12m, retct


In [ ]:
res_ct_df  = pd.concat(res_ct)
res_ct_df = res_ct_df[res_ct_df.index != 'alpha_i_t']
res_ct_df.head()

In [ ]:
res_chg = []
for var in vals:
    regdf = mdf5[['jdate', 'permno', 'ticker', 'ret_chg', var]].replace([np.inf, -np.inf], np.nan).dropna()
    print(f'{var}, retchg')
    results = fama_macbeth_reg_panel_nw(regdf, xname=var, yname='ret_chg')
#     print(results)
    res_chg.append(results)
#     print('')

In [ ]:
res_chg_df  = pd.concat(res_chg)

In [ ]:
res_chg_df = res_chg_df[res_chg_df.index != 'alpha_i_t']
res_chg_df.head()

In [ ]:
res_df = res_df.rename(columns={"mean":"mean_ret", "std_error":"std_error_ret",  "tstat":"tstat_ret",
                               "std_error_nw":"std_error_nw_ret",  "tstat_nw":"tstat_nw_ret"})
res_ct_df = res_ct_df.rename(columns={"mean":"mean_ct", "std_error":"std_error_ct",  "tstat":"tstat_ct",
                                     "std_error_nw":"std_error_nw_ct",  "tstat_nw":"tstat_nw_ct"})
res_chg_df = res_chg_df.rename(columns={"mean":"mean_chg", "std_error":"std_error_chg",  "tstat":"tstat_chg",
                                       "std_error_nw":"std_error_nw_chg",  "tstat_nw":"tstat_nw_chg"})

In [ ]:
res_df = res_df[["mean_ret", "std_error_nw_ret", "tstat_nw_ret"]]
res_ct_df = res_ct_df[["mean_ct", "std_error_nw_ct", "tstat_nw_ct"]]
res_chg_df = res_chg_df[["mean_chg", "std_error_nw_chg", "tstat_nw_chg"]]

In [ ]:
mm1 = res_df.merge(res_ct_df, left_index=True, right_index=True)
mm2 = mm1.merge(res_chg_df, left_index=True, right_index=True)

In [ ]:
mm2.head()

In [ ]:
mm2.to_csv('gkx_on_decomp_dcf5y_PLS_1970_2019.csv')